In [17]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [45]:
# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = reactor_nr
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]
ext_time_corr_df

The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,Reactor,0,1,2,4,6,8,10,15
0,1,00:00:00,01:00:20,02:00:40,04:01:05,06:00:30,08:01:00,10:00:40,15:01:00
1,2,00:00:00,01:00:20,02:00:40,04:01:05,06:00:30,08:01:00,10:00:40,15:01:00
2,3,00:03:55,01:04:25,02:04:05,04:04:40,06:03:50,08:05:25,10:05:05,15:05:55
3,4,00:03:55,01:04:25,02:04:05,04:04:40,06:03:50,08:05:25,10:05:05,15:05:55
4,5,00:07:00,01:07:55,02:07:30,04:08:35,06:07:15,08:10:50,10:09:30,15:12:00
5,6,00:07:00,01:07:55,02:07:30,04:08:35,06:07:15,08:10:50,10:09:30,15:12:00
6,7,00:11:15,01:11:30,02:11:05,04:12:20,06:10:50,08:15:25,10:14:00,15:17:30
7,8,00:11:15,01:11:30,02:11:05,04:12:20,06:10:50,08:15:25,10:14:00,15:17:30
8,9,00:14:40,01:15:05,02:14:30,04:16:25,06:14:15,08:20:20,10:18:25,15:22:55
9,10,00:14:40,01:15:05,02:14:30,04:16:25,06:14:15,08:20:20,10:18:25,15:22:55


In [19]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)



['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [34]:
srt.df.columns
ext_time_corr_df["Reactor"]

0                       1
1                       2
2                       3
3                       4
4                       5
5                       6
6                       7
7                       8
8                       9
9                      10
10                     11
11                     12
12                     13
13                     14
14                     15
15    closing of reactors
Name: Reactor, dtype: object

In [31]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame({"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list})
    kinetic_curve_entries["experiment"] = polymerisation_kinetic["Experiment number"]+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [32]:
kinetic_curves_df
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

,conversion,time,experiment
t0h-conversion,0,0,MRG-046-G_6_A
t1h-conversion,0.086971,1,MRG-046-G_6_A
t2h-conversion,0.104365,2,MRG-046-G_6_A
t4h-conversion,0.205776,4,MRG-046-G_6_A
t8h-conversion,0.33213,8,MRG-046-G_6_A
...,...,...,...
t1h-conversion,0.298969,1,MRG-046-G_10_E
t2h-conversion,0.436426,2,MRG-046-G_10_E
t4h-conversion,1.0,4,MRG-046-G_10_E
t8h-conversion,1.0,8,MRG-046-G_10_E


In [22]:
# and fit curve and the derivation
# and find the maximum of the derivation
# intersect/superimpose all curves

'''
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

# ToDo: check if this is the correct derivative
def sigmoid_derivative(x, L ,x0, k, b):
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y


fig = px.line(title="kinetic curve fit", labels={"x":"time", "y":"conversion"})
for idx, kinetic_curve in enumerate(kinetic_curves):
    # choosing a few test function
    if not idx in (1, 2, 3):
    # if not idx in (39, 40):
        continue

    ydata = kinetic_curve["conversion"].values.astype(float)
    xdata = kinetic_curve["time"].values

    p0 = [max(ydata), 1] # this is a mandatory initial guess
    popt, pcov = curve_fit(f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10)
    print(idx)

    # plot the fit and data
    extended_xdata = np.linspace(min(xdata)-2, max(xdata)+2, 100)
    y_extended = neg_growth(extended_xdata, *popt)
    notnan_extended = ~(np.isnan(extended_xdata) | np.isnan(y_extended))
    extended_xdata = extended_xdata[notnan_extended]
    y_extended = y_extended[notnan_extended]
#    extended_xdata = np.array(list(range(min(xdata)-2, min(xdata)-1, 1)) + xdata + list(range(max(xdata)+1, max(xdata)+3, 1)))
    fig.add_scatter(x=xdata, y=ydata, mode="lines", opacity=0.5)
    fig.add_scatter(x=extended_xdata, y=neg_growth(extended_xdata, *popt), mode="lines", opacity=0.5)
    fig.add_scatter(x=extended_xdata, y=neg_growth_derivative(extended_xdata, *popt), mode="lines", opacity=0.5)
    # print(neg_growth(extended_xdata, *popt))
    # break
fig.update_layout(
    yaxis=dict(
        range=[-0.5,1]
    ),
    xaxis_title="time",
    yaxis_title="conversion"
)
fig.show()



1
2
3


In [23]:
# get the maximum of the derivative
print(len(kinetic_curves))

330


In [24]:
ttpm1= 1, 1
ttpm2= 2, 1
ttpm4= 1, 2

xrange = np.arange(-0.5, 0.5, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm4):
    newfig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams), mode="lines", opacity=1)
#    newfig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5)

newfig.show()

In [25]:
ttpm1= 1, 1
ttpm2= 2, 1
ttpm3= 1, 2

xrange = np.arange(-0.5, 5, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm3):
    newfig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams), mode="lines")
    newfig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

newfig.show()